# 小百合JobExpress板块信息
+ 储存到阿里云mysql中

In [1]:
from requests_html import HTMLSession
import regex as re
import string
import time
from mylog import MyLog as mylog
import pymysql
import pickle
from tqdm import tqdm

In [2]:
SUCCESSES = 0
ERRORS = 0

In [3]:
logger = mylog()
htmlsession = HTMLSession()
headers = {
    'user-agent':
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'accept':
    'text/html, application/xhtml+xml, application/xml; q=0.9, image/webp, */*; q=0.8',
    'referer':
    'http://bbs.nju.edu.cn/g/JobExpress'
}

In [4]:
start_url = 'http://bbs.nju.edu.cn/bbsdoc?board=JobExpress'
pattern_index = re.compile('bbscon\?board=JobExpress&file=M\.(\d+)\.A&num=(\d+)')

In [5]:
keep_going = True
count_articles = 999999
url_articles = []

In [6]:
db = pymysql.connect("39.108.157.74","root","00genius00","JobExpress" )
cursor = db.cursor()

In [7]:
cursor.execute("SELECT MAX(id) FROM links")

1

In [8]:
count_crawled = cursor.fetchone()[0]
count_crawled

In [13]:
count_crawled -= 100
count_crawled

11914

In [14]:
def update_url_articles(res):
    global count_articles
    global keep_going
    global pattern_index
    for link in res.html.links:
        if link.startswith('bbscon?board=JobExpress&file=M.'):
            index = int(pattern_index.match(link).group(2))
            file = int(pattern_index.match(link).group(1))
            if index > count_crawled:
                count_articles = index if count_articles > index else count_articles
                url_articles.append((str(file), str(index), 'http://bbs.nju.edu.cn/'+link))
            else:
                keep_going = False

In [16]:
res = htmlsession.get(start_url, headers=headers)
update_url_articles(res)

In [17]:
while keep_going:
    count_articles = count_articles-20 if count_articles>=20 else 0
    target_url = 'http://bbs.nju.edu.cn/bbsdoc?board=JobExpress&start=%s&type=doc' %str(count_articles)
    res = htmlsession.get(target_url, headers=headers)
    update_url_articles(res)
    time.sleep(0.5)

In [18]:
def insert(cursor, db, file, index, url):
    global SUCCESSES, ERRORS
    sql_query = "REPLACE INTO links VALUES ('%s', '%s', '%s')" %(str(file), str(index), url)
    try:
        cursor.execute(sql_query)
        db.commit()
        logger.info('成功: 插入id为: %s 的招聘信息' %str(index))
        SUCCESSES += 1
    except:
        db.rollback()
        logger.error('失败: 插入id为: %s 的招聘信息' %str(index))
        ERRORS += 1

In [ ]:
for item in tqdm(url_articles):
    insert(cursor, db, item[0], item[1], item[2])

  0%|                                                                           | 0/40 [00:00<?, ?it/s]2018-08-28 16:01:34,295 INFO     ydzhang    成功: 插入id为: 12014 的招聘信息

  2%|█▋                                                                 | 1/40 [00:00<00:04,  9.27it/s]2018-08-28 16:01:34,401 INFO     ydzhang    成功: 插入id为: 12009 的招聘信息

  5%|███▎                                                               | 2/40 [00:00<00:04,  9.35it/s]2018-08-28 16:01:34,545 INFO     ydzhang    成功: 插入id为: 11998 的招聘信息

  8%|█████                                                              | 3/40 [00:00<00:04,  8.40it/s]2018-08-28 16:01:34,656 INFO     ydzhang    成功: 插入id为: 11997 的招聘信息

 10%|██████▋                                                            | 4/40 [00:00<00:04,  8.55it/s]2018-08-28 16:01:34,764 INFO     ydzhang    成功: 插入id为: 12002 的招聘信息

 12%|████████▍                                                          | 5/40 [00:00<00:04,  8.68it/s]2018-08-28 16:01:34,880 INFO     ydzhang  

In [23]:
print(SUCCESSES, ERRORS)

30 30
